In [ ]:
import numpy as np
from numpy import asarray
from numpy import save
from numpy import load

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, GlobalAvgPool2D
from tensorflow.keras.layers import Add, ReLU, Dense
from tensorflow.keras import Model
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import glob
import random

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# Loading data
X_train = load('../input/gnr-contest/X_train_augmented.npy')
Y_train = load('../input/gnr-contest/Y_train_augmented.npy')

In [ ]:
print(X_train.shape,Y_train.shape)
print(X_train[a].shape)

In [ ]:
Y_train2[0] = np.argmax(Y_train[0])+1
print(Y_train2.shape)

In [ ]:
a =310
plt.imshow(X_train[a]/255)
print(Y_train[a])

In [ ]:
#Conv-BatchNorm-ReLU block

def conv_layer(x, filters, kernel_size, strides=1):
    
    x = Conv2D(filters=filters, kernel_size=kernel_size, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [ ]:
#Identity block

def res_identity(tensor, filters):
    
    x = conv_layer(tensor, filters=filters, kernel_size=1, strides=1)
    x = conv_layer(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    x = Add()([tensor,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [ ]:
#Projection block

def projection_block(tensor, filters, strides):
    
    #left stream
    x = conv_layer(tensor, filters=filters, kernel_size=1, strides=strides)
    x = conv_layer(x, filters=filters, kernel_size=3, strides=1)
    x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)
    x = BatchNormalization()(x)
    
    #right stream
    shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([shortcut,x])    #skip connection
    x = ReLU()(x)
    
    return x

In [ ]:
#Resnet block

def resnet_block(x, filters, reps, strides):
    
    x = projection_block(x, filters, strides)
    for _ in range(reps-1):
        x = res_identity(x,filters)
        
    return x

In [ ]:
def get_model(model_name):
  if model_name == "ResNet":
    inputs = keras.Input(shape=(256, 256, 3), name="img")
    x = conv_layer(inputs, filters=64, kernel_size=7, strides=2)
    x = MaxPool2D(pool_size = 3, strides =2)(x)
    x = resnet_block(x, filters=32, reps =3, strides=1)
    x = resnet_block(x, filters=64, reps =2, strides=2)
    x = resnet_block(x, filters=128, reps =3, strides=2)
    x = resnet_block(x, filters=256, reps =3, strides=2)
    x = GlobalAvgPool2D()(x)
    x = Dense(256,activation='relu')(x)
    outputs = Dense(10, activation ='softmax')(x)

    model = keras.Model(inputs, outputs, name="Resnet")
    model.summary()
    return model
    
  elif model_name == "ResNet-18":
    inputs = keras.Input(shape=(256, 256, 3), name="img")
    x = conv_layer(inputs, filters=64, kernel_size=7, strides=2)
    x = MaxPool2D(pool_size = 3, strides =2)(x)
    x = resnet_block(x, filters=64, reps =2, strides=1)
    x = resnet_block(x, filters=128, reps =2, strides=2)
    x = resnet_block(x, filters=256, reps =2, strides=2)
    x = resnet_block(x, filters=512, reps =2, strides=2)
    x = GlobalAvgPool2D()(x)
    x = Dense(100,activation='relu')(x)
    outputs = Dense(10, activation ='softmax')(x)

    model = keras.Model(inputs, outputs, name="Resnet")
    model.summary()
    return model
    
  elif model_name == 'VGG':
    inputs = tf.keras.layers.Input((256, 256, 3))
    x = layers.Conv2D(32, 3, activation='relu', padding="same")(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding="same")(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding="same")(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding="same")(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding="same")(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding="same")(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(10,activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="VGG")

    model.summary()
    return model

  elif model_name == "VGG2":
    inputs = keras.Input(shape=(256, 256, 3), name="img")
    x = layers.Conv2D(32, 3, activation="relu")(inputs)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    x = layers.MaxPooling2D(3)(x)

    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)

    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(3)(x)

    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)

    x = layers.Conv2D(128, 3, activation="relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10,activation="softmax")(x)

    model = keras.Model(inputs, outputs, name="Mini_VGGNet")
    model.summary()
    return model
  """elif model_name == "Inception v2":
  elif model_name == "Inception v3": 
  elif model_name == "Inception v4":  
  elif model_name == "DenseNet":
  elif model_name == "MobileNet":"""
  


In [ ]:
model_name = "ResNet-18"
model = get_model(model_name)
fig_name = model_name +".png"
#keras.utils.plot_model(model, fig_name, show_shapes=True)

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/GNR model/Resnet_v2')

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy', 
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True)

In [ ]:
#plotting the learning curve
import pandas as pd
pd.DataFrame(history.history).plot()

In [ ]:
model.save('./Resnet-18')

In [ ]:
score = model.evaluate(X_train,Y_train,verbose=0)
print(score)

In [ ]:
prediction = model.predict(X_train)

In [ ]:
a =56
print(Y_train[a])
print(prediction[a])
print(np.argmax(prediction[a])+1)

In [ ]:
# prediction
images = load('../input/test-data/X_test.npy')
print("Shape of images: ",images.shape)
print("Number of images in class: ",images.shape[0])
print(images.shape)

In [ ]:
predictions = model.predict(images)
final_class_preds = []
no_images = images.shape[0]
for i in range(no_images):
  final_class_preds.append(np.argmax(predictions[i])+1)
print(len(final_class_preds))

In [ ]:
print(final_class_preds)

In [ ]:
if final_class_pred==final_class_preds:
    print(True)

In [ ]:
#creating csv file
import csv
import pandas

f = open('./200050103.csv', 'w')
row = 'ImageID','LabelID'

# create the csv writer
writer = csv.writer(f)
writer.writerow(row)
for i in range(101,201):
  row = i,final_class_preds[i-101]
  # write a row to the csv file
  writer.writerow(row)

# close the file
f.close()